In [13]:
%pwd

'/Users/srishanthreddy/medical-assistance-chatbot'

In [14]:
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter